In [1]:
import numpy as np
import networkx as nx
import time
import math
import pandas as pd
import scipy as sp
import plotly.express as px
import plotly.graph_objs as go
from scipy.sparse import *
from scipy import linalg
from scipy.sparse.linalg import norm
from scipy.optimize import least_squares

This function is needed in the algorithm. Note that this is a NON-functioning version, for now it's just a place holder. When algo2_testing will be completed, this will be updated and I'll work on algo4_testing.

In [2]:
def Arnoldi(A,v0,m):
    v = v0
    beta = norm(v)
    v = v/beta
    H = sp.sparse.lil_matrix((m+1,m)) 
    V = sp.sparse.lil_matrix((A.shape[0],m+1))
    V[:,0] = v # each column of V is a vector v

    for j in range(m):
        # print("j = ", j)
        w = A @ v  
        for i in range(j):
            tmp = v.T @ w # tmp is a 1x1 matrix, so it's O(1) in memory
            H[i,j] = tmp[0,0] 
            w = w - H[i,j]*v 
            
        H[j+1,j] = norm(w)

        if H[j+1,j] == 0:
            # print("Arnoldi breakdown")
            m = j
            v = 0
            break
        else:
            if j < m-1:
                v = w/H[j+1,j]
                V[:,j+1] = v

    # print(j, " iterations completed")
    # print("V = ", V.shape)
    # print("H = ", H.shape)
    # print("v = ", v.shape)
    # print("beta = ", beta)

    return V, H, v, beta, j  

## Algorithm 4 testing

Still a complete mess. Conceptually and technically wrong. I'll work on it when algo2_testing will be completed.

In [15]:
def Algo4(Pt, v, m, a: list, tau, maxit: int, x):
    
    iter = 1
    mv = 0
    I = sp.sparse.eye(n, n, format='lil')
    r = sp.sparse.lil_matrix((n,1))
    res = np.zeros(len(a)) 

    H_e1 = np.zeros((m+1,1))
    H_e1[0] = 1

    V_e1 = np.zeros((n,1))
    V_e1[0] = 1

    s_e1 = np.zeros((len(a),1))
    s_e1[0] = 1


    def find_k(res):
        k = 0
        for i in range(len(a)):
            if res[i] == max(res):
                k = i
                break
        return k

    def compute_gamma(res, a, k):
        gamma = sp.sparse.lil_matrix((len(a),1))
        for i in range(len(a)):
            if i != k:
                gamma[i] = (res[i]*a[k])/(res[k]*a[i])
            else:
                gamma[i] = 0
        return gamma

    # compute the residual vector
    for i in range(len(a)):
        r = ((1-a[i])/a[i])*v - ((1/a[i])*I - Pt) @ x
        res[i] = a[i]*norm(r)

    while max(res) >= tau and iter <= maxit:
        k = find_k(res)
        gamma = compute_gamma(res, a, k)
        # run Arnoldi, with a[k] as the shift
        V, H, v, beta, j = Arnoldi((1/a[k])*I - Pt, r, m)

        mv = mv + j

        # compute y as the minimizer of || beta*e1 - Hy ||_2 using the least squares method
        y = sp.sparse.linalg.lsqr(H, beta*H_e1)[0]
        tmp = V[:,0:y.shape[0]]

        # reshape y to be a column vector
        y = y.reshape(y.shape[0],1)
        x += tmp @ y

        # compute the residual vector
        res[k] = a[k]*np.linalg.norm(beta*V_e1 - tmp @ y)
        
        # for i in range(len(a)) but not k
        for i in range(len(a)):
            if i != k and res[i] >= tau:
                H = H + ((1-a[i])/a[i] - (1-a[k])/a[k])*sp.sparse.eye(H.shape[0], H.shape[1], format='lil')

                # solve y and gamma[i] from [H z] = [y gamma[i]]^T = gamma[i]*e1*beta
                z = beta*H_e1 - H.dot(y)
                A_tmp = sp.sparse.hstack([H, z])    
                b_tmp = gamma * beta 

                # make b_tmp a column vector
                b_tmp = b_tmp.reshape(b_tmp.shape[0],1)
                # add zeros to the end of b_tmp until it has the same number of rows as A_tmp
                b_tmp = sp.sparse.vstack([b_tmp, sp.sparse.lil_matrix((A_tmp.shape[0]-b_tmp.shape[0],1))])


                # solve the system, without using the least squares method
                # CONVERT TO CSC FORMAT TO USE SPARSE SOLVERS
                A_tmp = A_tmp.tocsc()

                result = sp.sparse.linalg.spsolve(A_tmp, b_tmp)[0]
                print("result:", result.shape)
                                
                
                # split the result into y and gamma
                y = result[0:y.shape[0]]
                gamma = result[y.shape[0]:]

                # update x
                x = x + tmp @ y
                # update residual
                res = (a[i]/a[k])*res*gamma[k]

        iter = iter + 1

In [16]:
# test the algorithm
n = 100
m = 110
maxit = 100
tau = 1e-6
a = [0.85, 0.9, 0.95, 0.99]
x = sp.sparse.lil_matrix((n,1))
x[0,0] = 1
# generate a random graph
G = nx.gnp_random_graph(n, 0.1, seed=1, directed=True)
# generate a random matrix
A = nx.adjacency_matrix(G)
# generate a random vector
v = sp.sparse.rand(n, 1, density=0.1, format='lil')
# compute the power iteration matrix
Pt = A.T
# run the algorithm
Algo4(Pt, v, m, a, tau, maxit, x)

/tmp/ipykernel_264197/102804730.py:12: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


result: ()


IndexError: invalid index to scalar variable.